VoiceCraft Inference Text To Speech Demo
===

### Select 'voicecraft' as the kernel

In [1]:
# Импорт библиотек
# Если тут ошибка, то установилось не правильно или нужно сменить ядро сверху!
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["USER"] = "me" # Cмените на свое имя пользователя

import torch
import torchaudio
import numpy as np
import random
from argparse import Namespace

from data.tokenizer import (
    AudioTokenizer,
    TextTokenizer,
)
from huggingface_hub import hf_hub_download

In [2]:
# # Установка MFA моделей и словарей если их еще нет(в докере должно быть)
# !source ~/.bashrc && \
#     conda activate voicecraft && \
#     mfa model download dictionary english_us_arpa && \
#     mfa model download acoustic english_us_arpa

In [3]:
# Установка MFA моделей и словарей если их еще нет
# # Загрузка модели, и проч
device = "cuda" if torch.cuda.is_available() else "cpu"
voicecraft_name="830M_TTSEnhanced.pth" # or giga330M.pth, 330M_TTSEnhanced.pth, giga830M.pth

from models import voicecraft
model = voicecraft.VoiceCraft.from_pretrained(f"pyp1/VoiceCraft_{voicecraft_name.replace('.pth', '')}")
phn2num = model.args.phn2num
config = vars(model.args)
model.to(device)

encodec_fn = "./pretrained_models/encodec_4cb2048_giga.th"
if not os.path.exists(encodec_fn):
    os.system(f"wget https://huggingface.co/pyp1/VoiceCraft/resolve/main/encodec_4cb2048_giga.th")
    os.system(f"mv encodec_4cb2048_giga.th ./pretrained_models/encodec_4cb2048_giga.th")
audio_tokenizer = AudioTokenizer(signature=encodec_fn, device=device)

text_tokenizer = TextTokenizer(backend="espeak")


Dora directory: /tmp/audiocraft_me


In [4]:
# Подготовка аудио
# Аудио, речь которого хотите склонировать
# Аудио, речь которого хотите склонировать
orig_audio = "./demo/5895_34622_000026_000002.wav"
orig_transcript = "Gwynplaine had, besides, for his work and for his feats of strength, round his neck and over his shoulders, an esclavine of leather."

# Аудио, речь которого хотите склонировать
temp_folder = "./demo/temp"
os.makedirs(temp_folder, exist_ok=True)
os.system(f"cp {orig_audio} {temp_folder}")
filename = os.path.splitext(orig_audio.split("/")[-1])[0]
with open(f"{temp_folder}/{filename}.txt", "w") as f:
    f.write(orig_transcript)
align_temp = f"{temp_folder}/mfa_alignments"
!source ~/.bashrc && \
    conda activate voicecraft && \
    mfa align -v --clean -j 1 --output_format csv {temp_folder} \
        english_us_arpa english_us_arpa {align_temp}

# # Если выше произойдет неудача,возможно аудио слишком сложное для модели, увеличение beam size может решить проблему
# !source ~/.bashrc && \
#     conda activate voicecraft && \
#     mfa align -v --clean -j 1 --output_format csv {temp_folder} \
#         english_us_arpa english_us_arpa {align_temp} --beam 1000 --retry_beam 2000



In [ ]:
# Смотрим на демо файл demo/temp/mfa_alignment, решаем какую часть аудио использовать для ввода
cut_off_sec = 3.6 # NOTE:согласно файлу demo/temp/mfa_alignments/5895_34622_000026_000002.wav, слово "strength" заканчивается 3.561 sec, так что используем 3.6 sec как ввод. Для каждого аудиофайла свое значение
target_transcript = "Gwynplaine had, besides, for his work and for his feats of strength, I cannot believe that the same model can also do text to speech synthesis too!"
# Заметка: 3 секунды записи обычно достаточно для высококачественного клонирования голоса, но чем длиннее, тем лучше (3-6с).
audio_fn = f"{temp_folder}/{filename}.wav"
info = torchaudio.info(audio_fn)
audio_dur = info.num_frames / info.sample_rate

assert cut_off_sec < audio_dur, f"cut_off_sec {cut_off_sec} is larger than the audio duration {audio_dur}"
prompt_end_frame = int(cut_off_sec * info.sample_rate)

# Запуск модели для получения вывода
# Гиперпараметры
codec_audio_sr = 16000
codec_sr = 50
top_k = 0
top_p = 0.9 # Можно попробовать 0.8, но 0.9 обычно лучше
temperature = 1
silence_tokens=[1388,1898,131]
kvcache = 1 

# Поменяйте 3 настройки снизу если плохо сгенерировалось 
stop_repetition = 3 # Если модель генерирует длинное молчание, уменьшить stop_repetition до 3,2 или 1
sample_batch_size = 3 # Если долгое молчание или ненатурально растянутые слова,увеличить sample_batch_size до 4 или выше. Это позволит модели запускать sample_batch_size примеры того же аудио и выбирать самый короткий.Так что если скорость речи слишком высокая - уменьшите значение.
seed = 1 # Поменяйте seed,если результат все еще не удовлетворяет

def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
seed_everything(seed)

decode_config = {'top_k': top_k, 'top_p': top_p, 'temperature': temperature, 'stop_repetition': stop_repetition, 'kvcache': kvcache, "codec_audio_sr": codec_audio_sr, "codec_sr": codec_sr, "silence_tokens": silence_tokens, "sample_batch_size": sample_batch_size}
from inference_tts_scale import inference_one_sample
concated_audio, gen_audio = inference_one_sample(model, Namespace(**config), phn2num, text_tokenizer, audio_tokenizer, audio_fn, target_transcript, device, decode_config, prompt_end_frame)
        
# сохранение фрагментов для сравнения
concated_audio, gen_audio = concated_audio[0].cpu(), gen_audio[0].cpu()
# logging.info(f"length of the resynthesize orig audio: {orig_audio.shape}")


# Отображение аудио
from IPython.display import Audio
print("concatenate prompt and generated:")
display(Audio(concated_audio, rate=codec_audio_sr))

print("generated:")
display(Audio(gen_audio, rate=codec_audio_sr))

# # СОхранение аудио
# # output_dir
# output_dir = "/home/pyp/VoiceCraft/demo/generated_tts"
# os.makedirs(output_dir, exist_ok=True)
# seg_save_fn_gen = f"{output_dir}/{os.path.basename(audio_fn)[:-4]}_gen_seed{seed}.wav"
# seg_save_fn_concat = f"{output_dir}/{os.path.basename(audio_fn)[:-4]}_concat_seed{seed}.wav"        

# torchaudio.save(seg_save_fn_gen, gen_audio, codec_audio_sr)
# torchaudio.save(seg_save_fn_concat, concated_audio, codec_audio_sr)

# Может быть предупреждение WARNING:phonemizer:words count mismatch on 300.0% of the lines (3/1), Его можно игнорировать